In [28]:
import pyfair

In [29]:
import numpy as np
import pandas as pd

import matplotlib
%matplotlib inline

pd.options.display.float_format = '{:,.2f}'.format

In [30]:
from pyfair import FairModel
from pyfair import FairMetaModel
from pyfair import FairSimpleReport


europe_attacks = FairModel(name='European Attacks', n_simulations=5)
europe_attacks.input_data('lef', mean=1_000, stdev=100)
europe_attacks.input_data('lm', mean=10_000, stdev=5_000)
europe_attacks.calculate_all()

ap_attacks = FairModel(name='Asia-Pacific Attacks', n_simulations=5)
ap_attacks.input_data('lef', mean=3_000, stdev=100)
ap_attacks.input_data('lm', mean=10_000, stdev=5_000)
ap_attacks.calculate_all() 

independent_meta = FairMetaModel('Indepedent Metamodel', models=[europe_attacks, ap_attacks])
independent_meta.calculate_all()

In [31]:
ap_results = ap_attacks.export_results()[['Risk', 'Loss Event Frequency', 'Loss Magnitude']]
ap_results.columns = ['AP ' + column for column in ap_results.columns]
europe_results = europe_attacks.export_results()[['Risk', 'Loss Event Frequency', 'Loss Magnitude']]
europe_results.columns = ['Europe ' + column for column in europe_results.columns]

In [32]:
ap_results

,AP Risk,AP Loss Event Frequency,AP Loss Magnitude
0,"26,926,510.23","3,049.67","8,829.32"
1,"53,440,753.55","2,986.17","17,896.06"
2,"42,407,738.81","3,064.77","13,837.17"
3,"24,123,402.31","3,152.30","7,652.63"
4,"37,840,726.15","2,976.58","12,712.80"


In [33]:
europe_results

,Europe Risk,Europe Loss Event Frequency,Europe Loss Magnitude
0,"9,267,879.80","1,049.67","8,829.32"
1,"17,648,625.40",986.17,"17,896.06"
2,"14,733,391.52","1,064.77","13,837.17"
3,"8,818,146.17","1,152.30","7,652.63"
4,"12,415,125.71",976.58,"12,712.80"


In [34]:
independent_results = pd.DataFrame({
    'AP Risk': ap_results['AP Risk'],
    'Europe Risk': europe_results['Europe Risk'],
    'Aggregate': ap_results['AP Risk'] + europe_results['Europe Risk']
})

independent_results

,AP Risk,Europe Risk,Aggregate
0,"26,926,510.23","9,267,879.80","36,194,390.03"
1,"53,440,753.55","17,648,625.40","71,089,378.95"
2,"42,407,738.81","14,733,391.52","57,141,130.34"
3,"24,123,402.31","8,818,146.17","32,941,548.49"
4,"37,840,726.15","12,415,125.71","50,255,851.86"


$ Risk(Aggregate) = Risk(AP) + Risk(Europe | AP)$

In [35]:
ap_attacks2 = FairModel(name='Asia-Pacific Attacks', n_simulations=5)
ap_attacks2.input_data('lef', mean=3_000, stdev=100)
ap_attacks2.input_data('lm', mean=10_000, stdev=5_000)
ap_attacks2.calculate_all() 

europe_attacks2 = FairModel(name='European Attacks', n_simulations=5)
europe_attacks2.input_data('lef', mean=1_000, stdev=100)
europe_attacks2.input_data('lm', mean=7_500, stdev=5_000)
europe_attacks2.calculate_all()


dependent_meta = FairMetaModel('Dependent Metamodel', models=[europe_attacks2, ap_attacks2])
dependent_meta.calculate_all()

In [37]:
ap_results2 = ap_attacks2.export_results()[['Risk', 'Loss Event Frequency', 'Loss Magnitude']]
ap_results2.columns = ['AP ' + column for column in ap_results2.columns]
europe_results2 = europe_attacks2.export_results()[['Risk', 'Loss Event Frequency', 'Loss Magnitude']]
europe_results2.columns = ['Europe ' + column for column in europe_results2.columns]

In [38]:
ap_results2

,AP Risk,AP Loss Event Frequency,AP Loss Magnitude
0,"26,926,510.23","3,049.67","8,829.32"
1,"53,440,753.55","2,986.17","17,896.06"
2,"42,407,738.81","3,064.77","13,837.17"
3,"24,123,402.31","3,152.30","7,652.63"
4,"37,840,726.15","2,976.58","12,712.80"


In [39]:
europe_results2

,Europe Risk,Europe Loss Event Frequency,Europe Loss Magnitude
0,"6,643,701.26","1,049.67","6,329.32"
1,"15,183,191.47",986.17,"15,396.06"
2,"12,071,469.39","1,064.77","11,337.17"
3,"5,937,388.71","1,152.30","5,152.63"
4,"9,973,664.05",976.58,"10,212.80"


In [40]:
dependent_results = pd.DataFrame({
    'AP Risk': ap_results2['AP Risk'],
    'Europe Risk': europe_results2['Europe Risk'],
    'Aggregate': ap_results2['AP Risk'] + europe_results2['Europe Risk']
})

dependent_results

,AP Risk,Europe Risk,Aggregate
0,"26,926,510.23","6,643,701.26","33,570,211.49"
1,"53,440,753.55","15,183,191.47","68,623,945.03"
2,"42,407,738.81","12,071,469.39","54,479,208.20"
3,"24,123,402.31","5,937,388.71","30,060,791.02"
4,"37,840,726.15","9,973,664.05","47,814,390.20"


In [1]:
import pyfair

In [4]:
model1 = pyfair.FairModel(name="Regular Model 1", n_simulations=10_000)
model1.input_data('Vulnerability', low=.1, mode=.2, high=.9)
model1.input_data('Threat Event Frequency', low=20, mode=100, high=900)
model1.input_data('Primary Loss', low=3_000_000, mode=3_500_000, high=5_000_000)
model1.input_data('Secondary Loss', constant=3_500_000)
model1.calculate_all()
model1.export_results().to_csv('model_output.csv')

# Create another model using LEF (Normal) and LM (PERT)
model2 = pyfair.FairModel(name="Regular Model 2", n_simulations=10_000)
model2.input_data('Loss Event Frequency', mean=.3, stdev=.1)
model2.input_data('Loss Magnitude', low=2_000_000_000, mode=3_000_000_000, high=5_000_000_000)
model2.calculate_all()

# Create metamodel by combining 1 and 2
mm = pyfair.FairMetaModel(name='My Meta Model!', models=[model1, model2])
mm.calculate_all()
mm.export_results().to_csv('mm_output.csv')



$ Risk(Agg) = Risk(AP) + Risk(EU | AP) + Risk(North America | EU | AP)$

In [ ]:
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

from matplotlib.patches import Patch
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection


class FairTreeGraph(object):
    '''Provides a pretty tree diagram to summarize calculations.
    
    '''
    
    # Class attribute
    DIMENSIONS = pd.DataFrame.from_dict({
    'Contact'                       : ['Contact'   ,    0,    0,  600,  800, 'green', None],
    'Threat Event Frequency'        : ['Threat\nEvent\nFrequency' ,  600,  800, 1800, 1600, 'blue', 'x'],
    'Action'                        : ['Action'   , 1200,    0,  600,  800, 'green', None],
    'Threat Capability'             : ['Threat\nCapability'  , 2400,    0, 3000,  800, 'gray', None],
    'Vulnerability'                 : ['Vulnerability'   , 3000,  800, 1800, 1600, 'gray', 'step'],
    'Control Strength'              : ['Control\nStrength'  , 3600,    0, 3000,  800, 'gray', None],
    'Loss Magnitude'                : ['Loss\nMagnitude'  , 6600, 1600, 4200, 2400, 'green', '+'],
    'Loss Event Frequency'          : ['Loss\nEvent\nFrequency', 1800, 1600, 4200, 2400, 'green', 'x'],
    'Risk'                          : ['Risk'   , 4200, 2400, 4200, 5000, 'blue', 'x'],
    'Primary Loss'                  : ['Primary\nLoss'  , 5400,  800, 6600, 1600, 'gray', None],
    'Secondary Loss'                : ['Secondary\nLoss'  , 7800,  800, 6600, 1600, 'gray', 'x'],
    'Secondary Loss Event Frequency': ['Secondary\nLoss Event\nFrequency', 7200,    0, 7800,  800, 'gray', None],
    'Secondary Loss Event Magnitude': ['Secondary\nLoss Event\nMagnitude', 8400,    0, 7800,  800, 'gray', None],
}, orient='index', columns=['tag', 'self_x', 'self_y', 'parent_x', 'parent_y', 'color', 'function'])
    
    def __init__(self):
        self._colormap = {'Not Required': 'grey', 'Supplied': 'green', 'Calculated': 'blue'}


    def _process_statuses(self):
        '''Turn dict into df and add color column'''
        self._statuses = pd.DataFrame.from_records([self._statuses]).T
        self._statuses.columns = ['status']
        self._statuses['color'] = self._statuses['status'].map(self._colormap)
        
    def _tweak_axes(self, ax):
        # Set limits
        ax.set_title('LEF and LM Example', fontsize=20)
        ax.set_xlim(0, 9_400)
        ax.set_ylim(0, 2_900)
        # Disappear axes and spines
        for axis in [ax.xaxis, ax.yaxis]:
            axis.set_visible(False)
        for spine_name in ['left', 'right', 'top', 'bottom']:
            ax.spines[spine_name].set_visible(False)
        return ax
    
    def _generate_rects(self, ax):
        '''Cannot be done via apply'''
        patches = []
        patch_colors = []
        for index, row in self.DIMENSIONS.iterrows():
            rect = Rectangle(
                (row['self_x'], row['self_y']),
                1000,
                500,
                alpha=.3,
            )
            patches.append(rect)
            patch_colors.append(row['color'])
        collection = PatchCollection(patches, facecolor=patch_colors, alpha=.3)
        ax.add_collection(collection)
        return ax
    
    def _generate_text(self, row, ax):
        '''Apply-able function'''
        # Draw header
        plt.text(
            row['self_x'] + 500, 
            row['self_y'] + 240, 
            row['tag'], 
            horizontalalignment='center',
            verticalalignment='center',
            fontsize=14,
            fontweight='medium',
        )

    def _generate_operators(self, row, ax):
        if row.color == 'blue':
            ax.text(
                row['self_x'] + 500, 
                row['self_y'] - 180,
                row['function'],
                horizontalalignment='center',
                verticalalignment='center',
                fontsize=15,
                fontweight='bold',
            )

    def _generate_lines(self, row, ax):
        '''Generate lines between boxes'''
        if row.color != 'gray' and row.name != 'Risk':
            ax.annotate(
                None,
                xy=(row['parent_x'] + 500, row['parent_y']), 
                xytext=(row['self_x'] + 500, row['self_y'] + 500),     
                arrowprops=dict(
                    arrowstyle="-",
                    connectionstyle="angle3,angleA=0,angleB=-90",
                    ec=row['color'],
                    alpha=.3,
                    linestyle='--', 
                    linewidth=3
                ),
            )
            
    def _generate_legend(self, ax):
        # Gen legend
        patches = [Patch(color=color, label=label, alpha=.3) for label, color in self._colormap.items()]
        plt.legend(handles=patches, frameon=False)

    def generate_image(self):
        fig, ax = plt.subplots()
        fig.set_size_inches(20,6)
        self.DIMENSIONS.apply(self._generate_lines, args=[ax], axis=1)
        ax = self._tweak_axes(ax)
        self.DIMENSIONS.apply(self._generate_text, args=[ax], axis=1)
        self.DIMENSIONS.apply(self._generate_operators, args=[ax], axis=1)
        self._generate_rects(ax)

        #ax.text(0, -500, 'Copyright 2019, Theo Naunheim\nFreely available for use under the CC BY 2.0 License')
        self._generate_legend(ax)
        return (fig, ax)

    
FairTreeGraph().generate_image()

In [ ]:
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

from matplotlib.patches import Patch
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection


class FairTreeGraph(object):
    '''Provides a pretty tree diagram to summarize calculations.
    
    '''
    '''<TDF, TC, CS, PL, SL EXAMPLE>'''
    # Class attribute
    DIMENSIONS = pd.DataFrame.from_dict({
    'Contact'                       : ['Contact'   ,    0,    0,  600,  800, 'gray', None],
    'Threat Event Frequency'        : ['Threat\nEvent\nFrequency' ,  600,  800, 1800, 1600, 'green', 'multiply'],
    'Action'                        : ['Action'   , 1200,    0,  600,  800, 'gray', None],
    'Threat Capability'             : ['Threat\nCapability'  , 2400,    0, 3000,  800, 'green', None],
    'Vulnerability'                 : ['Vulnerability'   , 3000,  800, 1800, 1600, 'blue', 'step'],
    'Control Strength'              : ['Control\nStrength'  , 3600,    0, 3000,  800, 'green', None],
    'Loss Magnitude'                : ['Loss\nMagnitude'  , 6600, 1600, 4200, 2400, 'blue', 'add'],
    'Loss Event Frequency'          : ['Loss\nEvent\nFrequency', 1800, 1600, 4200, 2400, 'blue', 'multiply'],
    'Risk'                          : ['Risk'   , 4200, 2400, 4200, 5000, 'blue', 'multiply'],
    'Primary Loss'                  : ['Primary\nLoss'  , 5400,  800, 6600, 1600, 'green', None],
    'Secondary Loss'                : ['Secondary\nLoss'  , 7800,  800, 6600, 1600, 'green', 'multiply'],
    'Secondary Loss Event Frequency': ['Secondary\nLoss Event\nFrequency', 7200,    0, 7800,  800, 'gray', None],
    'Secondary Loss Event Magnitude': ['Secondary\nLoss Event\nMagnitude', 8400,    0, 7800,  800, 'gray', None],
}, orient='index', columns=['tag', 'self_x', 'self_y', 'parent_x', 'parent_y', 'color', 'function'])
    
    def __init__(self):
        self._colormap = {'Not Required': 'grey', 'Supplied': 'green', 'Calculated': 'blue'}


    def _process_statuses(self):
        '''Turn dict into df and add color column'''
        self._statuses = pd.DataFrame.from_records([self._statuses]).T
        self._statuses.columns = ['status']
        self._statuses['color'] = self._statuses['status'].map(self._colormap)
        
    def _tweak_axes(self, ax):
        # Set limits
        ax.set_title('TEF, TC, CS, PL, and SL Example', fontsize=20)
        ax.set_xlim(0, 9_400)
        ax.set_ylim(0, 2_900)
        # Disappear axes and spines
        for axis in [ax.xaxis, ax.yaxis]:
            axis.set_visible(False)
        for spine_name in ['left', 'right', 'top', 'bottom']:
            ax.spines[spine_name].set_visible(False)
        return ax
    
    def _generate_rects(self, ax):
        '''Cannot be done via apply'''
        patches = []
        patch_colors = []
        for index, row in self.DIMENSIONS.iterrows():
            rect = Rectangle(
                (row['self_x'], row['self_y']),
                1000,
                500,
                alpha=.3,
            )
            patches.append(rect)
            patch_colors.append(row['color'])
        collection = PatchCollection(patches, facecolor=patch_colors, alpha=.3)
        ax.add_collection(collection)
        return ax
    
    def _generate_text(self, row, ax):
        '''Apply-able function'''
        # Draw header
        plt.text(
            row['self_x'] + 500, 
            row['self_y'] + 240, 
            row['tag'], 
            horizontalalignment='center',
            verticalalignment='center',
            fontsize=14,
            fontweight='medium',
        )


    def _generate_lines(self, row, ax):
        '''Generate lines between boxes'''
        if row.color != 'gray' and row.name != 'Risk':
            ax.annotate(
                None,
                xy=(row['parent_x'] + 500, row['parent_y']), 
                xytext=(row['self_x'] + 500, row['self_y'] + 500),     
                arrowprops=dict(
                    arrowstyle="-",
                    connectionstyle="angle3,angleA=0,angleB=-90",
                    ec=row['color'],
                    alpha=.3,
                    linestyle='--', 
                    linewidth=3
                ),
            )
            
    def _generate_legend(self, ax):
        # Gen legend
        patches = [Patch(color=color, label=label, alpha=.3) for label, color in self._colormap.items()]
        plt.legend(handles=patches, frameon=False)

    def generate_image(self):
        fig, ax = plt.subplots()
        fig.set_size_inches(20,6)
        self.DIMENSIONS.apply(self._generate_lines, args=[ax], axis=1)
        ax = self._tweak_axes(ax)
        self.DIMENSIONS.apply(self._generate_text, args=[ax], axis=1)
        self._generate_rects(ax)

        #ax.text(0, -500, 'Copyright 2019, Theo Naunheim\nFreely available for use under the CC BY 2.0 License')
        self._generate_legend(ax)
        return (fig, ax)

    
FairTreeGraph().generate_image()

In [ ]:
from pyfair import FairModel, FairSimpleReport


# Create our model and calculate (don't worry about understanding yet)
model = FairModel(name='Sample')
model.input_data('Threat Event Frequency', mean=50_000, stdev=10_000)
model.input_data('Vulnerability', p=.66)
model.input_data('Loss Magnitude', mean=100, stdev=50)
model.calculate_all()

FairSimpleReport(model).to_html('C:/Users/theon/Desktop/crap.html')

In [ ]:
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

from matplotlib.patches import Patch
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection


class FairTreeGraph(object):
    '''Provides a pretty tree diagram to summarize calculations.
    
    '''
    '''<TDF, TC, CS, PL, SL EXAMPLE>'''
    # Class attribute
    DIMENSIONS = pd.DataFrame.from_dict({
    'Contact'                       : ['Contact'   ,    0,    0,  600,  800, 'gray', None],
    'Threat Event Frequency'        : ['Threat\nEvent\nFrequency' ,  600,  800, 1800, 1600, 'gray', 'multiply'],
    'Action'                        : ['Action'   , 1200,    0,  600,  800, 'gray', None],
    'Threat Capability'             : ['Threat\nCapability'  , 2400,    0, 3000,  800, 'gray', None],
    'Vulnerability'                 : ['Vulnerability'   , 3000,  800, 1800, 1600, 'gray', 'step'],
    'Control Strength'              : ['Control\nStrength'  , 3600,    0, 3000,  800, 'gray', None],
    'Loss Magnitude'                : ['Loss\nMagnitude'  , 6600, 1600, 4200, 2400, 'gray', 'add'],
    'Loss Event Frequency'          : ['Loss\nEvent\nFrequency', 1800, 1600, 4200, 2400, 'green', 'multiply'],
    'Risk'                          : ['Risk'   , 4200, 2400, 4200, 5000, 'gray', 'multiply'],
    'Primary Loss'                  : ['Primary\nLoss'  , 5400,  800, 6600, 1600, 'gray', None],
    'Secondary Loss'                : ['Secondary\nLoss'  , 7800,  800, 6600, 1600, 'gray', 'multiply'],
    'Secondary Loss Event Frequency': ['Secondary\nLoss Event\nFrequency', 7200,    0, 7800,  800, 'gray', None],
    'Secondary Loss Event Magnitude': ['Secondary\nLoss Event\nMagnitude', 8400,    0, 7800,  800, 'gray', None],
}, orient='index', columns=['tag', 'self_x', 'self_y', 'parent_x', 'parent_y', 'color', 'function'])
    
    def __init__(self):
        self._colormap = {'Not Required': 'grey', 'Supplied': 'green', 'Calculated': 'blue'}


    def _process_statuses(self):
        '''Turn dict into df and add color column'''
        self._statuses = pd.DataFrame.from_records([self._statuses]).T
        self._statuses.columns = ['status']
        self._statuses['color'] = self._statuses['status'].map(self._colormap)
        
    def _tweak_axes(self, ax):
        # Set limits
        ax.set_title('Incomplete Example', fontsize=20)
        ax.set_xlim(0, 9_400)
        ax.set_ylim(0, 2_900)
        # Disappear axes and spines
        for axis in [ax.xaxis, ax.yaxis]:
            axis.set_visible(False)
        for spine_name in ['left', 'right', 'top', 'bottom']:
            ax.spines[spine_name].set_visible(False)
        return ax
    
    def _generate_rects(self, ax):
        '''Cannot be done via apply'''
        patches = []
        patch_colors = []
        for index, row in self.DIMENSIONS.iterrows():
            rect = Rectangle(
                (row['self_x'], row['self_y']),
                1000,
                500,
                alpha=.3,
            )
            patches.append(rect)
            patch_colors.append(row['color'])
        collection = PatchCollection(patches, facecolor=patch_colors, alpha=.3)
        ax.add_collection(collection)
        return ax
    
    def _generate_text(self, row, ax):
        '''Apply-able function'''
        # Draw header
        plt.text(
            row['self_x'] + 500, 
            row['self_y'] + 240, 
            row['tag'], 
            horizontalalignment='center',
            verticalalignment='center',
            fontsize=14,
            fontweight='medium',
        )


    def _generate_lines(self, row, ax):
        '''Generate lines between boxes'''
        if row.color != 'gray' and row.name != 'Risk':
            ax.annotate(
                None,
                xy=(row['parent_x'] + 500, row['parent_y']), 
                xytext=(row['self_x'] + 500, row['self_y'] + 500),     
                arrowprops=dict(
                    arrowstyle="-",
                    connectionstyle="angle3,angleA=0,angleB=-90",
                    ec=row['color'],
                    alpha=.3,
                    linestyle='--', 
                    linewidth=3
                ),
            )
            
    def _generate_legend(self, ax):
        # Gen legend
        patches = [Patch(color=color, label=label, alpha=.3) for label, color in self._colormap.items()]
        plt.legend(handles=patches, frameon=False)

    def generate_image(self):
        fig, ax = plt.subplots()
        fig.set_size_inches(20,6)
        self.DIMENSIONS.apply(self._generate_lines, args=[ax], axis=1)
        ax = self._tweak_axes(ax)
        self.DIMENSIONS.apply(self._generate_text, args=[ax], axis=1)
        self._generate_rects(ax)

        #ax.text(0, -500, 'Copyright 2019, Theo Naunheim\nFreely available for use under the CC BY 2.0 License')
        self._generate_legend(ax)
        return (fig, ax)

    
FairTreeGraph().generate_image()

In [ ]:
from pyfair.report.base_curve import FairBaseCurve
from pyfair.model.model import FairModel
from pyfair.model.meta_model import FairMetaModel

In [ ]:
fbc = FairBaseCurve()

In [ ]:
model = FairModel('model')
fbc._input_check([model, model])

In [ ]:
import warnings

warnings.filterwarnings("error")


meta = FairMetaModel('meta', models=[model, model])

In [ ]:
import tempfile
tf = tempfile.NamedTemporaryFile()
dir(tf)

with tempfile.NamedTemporaryFile() as tf:
    print(tf.name)
    
tf.delete

In [ ]:
import pandas as pd

import matplotlib
%matplotlib inline


from pyfair.utility.beta_pert import FairBetaPert

fbp = FairBetaPert(low=0, mode=10, high=20)
variates = pd.Series(fbp.random_variates(10_000))
variates.plot.hist(bins=100)

In [ ]:
import pathlib

p = pathlib.Path().home()
pathlib.Path(p)

In [ ]:
import sqlite3

In [ ]:
dir(sqlite3)